In [3]:
%pip install beautifulsoup4
%pip install pandas
%pip install requests
%pip install wordcloud
%pip install webdriver_manager
%pip install selenium 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)
Using cached charset_normalizer-3.3.2-cp310-cp310-win_amd64.whl (100 kB)
Using cached idna-3.6-py3-none-any.whl (61 kB)
Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached wordcloud-1.9.3-cp310-cp310-win_amd64.whl.metadata (3.5 kB)
  Using cached Pillow-10.1.0-cp310-cp310-win_amd64.whl.metadata (9.6 kB)
  Using cached matplotlib-3.

In [2]:
%pip install chromedriver-autoinstaller

  Using cached chromedriver_autoinstaller-0.6.3-py3-none-any.whl.metadata (2.0 kB)
Using cached chromedriver_autoinstaller-0.6.3-py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import math 
# Ensure the proper version of ChromeDriver is installed
chromedriver_autoinstaller.install()



'c:\\Users\\mudar\\.vscode\\apps\\works\\linkedin-skills\\.venv\\lib\\site-packages\\chromedriver_autoinstaller\\120\\chromedriver.exe'

https://medium.com/@alaeddine.grine/linkedin-job-scraper-and-matcher-85d0308ef9aa

scrape_job_details() gets the description of each job.

In [3]:
def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height

        time.sleep(sleep_time) 

In [5]:
keywords = "Data scientist"
location = "England"
start = 25


url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
List_Job_IDs = []

scroll_to_bottom(driver)
# 1. Get number of jobs found and number of pages:
try:
    div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
    number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
except:
    number_of_jobs = 0
    
number_of_pages=math.ceil(number_of_jobs/25)
print("number_of_jobs:",number_of_jobs)
print("number_of_pages:",number_of_pages)

# 2. Function to find Job Ids:
def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
    
    return Job_Ids_on_the_page  
  
 # 3. Get Job IDs that are on the first page:
Jobs_on_1st_page = find_Job_Ids(soup)
List_Job_IDs.extend(Jobs_on_1st_page)
job_title = "Data scientist"
location = "England"
# 4. Iterate over the remaining pages:
if number_of_pages>1:
    
    for page_num in range(1,number_of_pages):
        print(f"Scraping page: {page_num}",end="...")
        
        url = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&location={location}&start={25 * page_num}'
        url = requests.utils.requote_uri(url)
        driver.get(url)
        scroll_to_bottom(driver)

        # Parse the HTML content of the page using BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Get Job Ids present on the page.
        Jobs_on_this_page = find_Job_Ids(soup)
        List_Job_IDs.extend(Jobs_on_this_page) 

number_of_jobs: 358
number_of_pages: 15
Scraping page: 1...Scraping page: 2...Scraping page: 3...Scraping page: 4...Scraping page: 5...Scraping page: 6...Scraping page: 7...Scraping page: 8...Scraping page: 9...Scraping page: 10...Scraping page: 11...Scraping page: 12...Scraping page: 13...Scraping page: 14...

In [6]:

def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
  
job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
job={}
list_jobs=[]

for j in range(0,len(List_Job_IDs)):
    print(f"{j+1} ... read jobId:{List_Job_IDs[j]}")

    resp = requests.get(job_url.format(List_Job_IDs[j]))
    soup=BeautifulSoup(resp.text,'html.parser')
    # print(soup.prettify()) 

    job["Job_ID"] = List_Job_IDs[j] 
    
    try: 
        job["Job_txt"] = remove_tags(resp.content)
    except:
        job["Job_txt"] = None
    
    try:
        job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        job["company"]=None

    try:
        job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        job["job-title"]=None

    try:
        job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        job["level"]=None

    try:
        job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job["location"]=None

    try:
        job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job["posted-time-ago"]=None

    try:
        nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
        nb_candidats = int(nb_candidats.split()[0])
        job["nb_candidats"]= nb_candidats
    except:
        job["nb_candidats"]=None

    list_jobs.append(job)
    job={}

# create a pandas Datadrame
jobs_ds_england = pd.DataFrame(list_jobs)

1 ... read jobId:3686174962
2 ... read jobId:3756334739
3 ... read jobId:3781946078
4 ... read jobId:3738202754
5 ... read jobId:3789403299
6 ... read jobId:3779279757
7 ... read jobId:3792180741
8 ... read jobId:3686174962
9 ... read jobId:3756334739
10 ... read jobId:3781946078
11 ... read jobId:3738202754
12 ... read jobId:3789403299
13 ... read jobId:3779279757
14 ... read jobId:3792180741
15 ... read jobId:3766078688
16 ... read jobId:3742078999
17 ... read jobId:3790128372
18 ... read jobId:3781943412
19 ... read jobId:3790810848
20 ... read jobId:3785827286
21 ... read jobId:3756011819
22 ... read jobId:3789505102
23 ... read jobId:3793780041
24 ... read jobId:3788994791
25 ... read jobId:3794104425
26 ... read jobId:3784412071
27 ... read jobId:3790703596
28 ... read jobId:3756510937
29 ... read jobId:3782184667
30 ... read jobId:3771183710
31 ... read jobId:3658859360
32 ... read jobId:3789503387
33 ... read jobId:3788999877
34 ... read jobId:3792650856
35 ... read jobId:37895

In [7]:
len(jobs_ds_england)

123

In [8]:
for job in jobs_ds_england.Job_txt:
    print(job)


Sr Data Scientist SAS Marlow, England, United Kingdom 1 month ago 101 applicants See who SAS has hired for this role Apply Join or sign in to find your next job Join to apply for the Sr Data Scientist role at SAS Not you? Remove photo First name Last name Email Password (8+ characters) By clicking Agree & Join, you agree to the LinkedIn User Agreement , Privacy Policy and Cookie Policy . Continue Agree & Join or Apply on company website Security verification Close Already on LinkedIn? Sign in Save Save job Save this job with your existing LinkedIn profile, or create a new one. Your job seeking activity is only visible to you. Email Continue Welcome back Sign in to save Sr Data Scientist at SAS . Email or phone Password Show Forgot password? Sign in Report this job Sr Data Scientist – Hybrid Nice to meet you! We’re the leader in analytics. Through our software and services, we inspire customers around the world to transform data into intelligence – and questions into answers. We’re als

## PLACE ALL IN CLASSES

In [68]:
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import math 
# Ensure the proper version of ChromeDriver is installed
chromedriver_autoinstaller.install()

class LinkedInJobScraper:
    def __init__(self, job_title, location):
        self.job_title = job_title
        self.location = location
        self.List_Job_IDs = []
        self.driver = webdriver.Chrome()  # Change this to your preferred WebDriver
        self.user = 'cristinamudarrapradas@gmail.com'
        self.pwd = '532348290MaRiNaDoR!.'
        
    def log_in(self):
        # Open LinkedIn login page
        self.driver.get('https://www.linkedin.com/login')
        time.sleep(5)
        
        # Find the username and password fields by their IDs and input the credentials
        username_field = self.driver.find_element('id', 'username')
        password_field = self.driver.find_element('id', 'password')

        # Input your credentials
        username_field.send_keys(self.user)
        password_field.send_keys(self.pwd)

        # Submit the login form
        login_button = self.driver.find_element('xpath', '//button[@type="submit"]')
        login_button.click()

    def scroll_to_load_jobs(self, page_num=1, sleep_time=5):
        url = f'https://www.linkedin.com/jobs/search/?keywords={self.job_title}&location={self.location}&start={25 * (page_num - 1)}'
        self.driver.get(url)

        #self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        
        last_height = self.driver.execute_script('return document.body.scrollHeight')
        while True:
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            new_height = self.driver.execute_script('return document.body.scrollHeight')
            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(sleep_time) 


    def find_job_id_aux(self, soup):
        Job_Ids_on_the_page = []
        job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
        
        for job_posting in job_postings:
            Job_ID = job_posting.get('data-occludable-job-id')
            Job_Ids_on_the_page.append(Job_ID)
        
        return Job_Ids_on_the_page  
    
    
    
    def find_total_job_ids(self):
        print("Entered find_total_job_ids")
        # Parse the HTML content of the page using BeautifulSoup.
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        # Get number of jobs found and number of pages:
        try:
            div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
            number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
        except:
            number_of_jobs = 0
            
        number_of_pages=math.ceil(number_of_jobs/25)
        print("number_of_jobs:", number_of_jobs)
        print("number_of_pages:", number_of_pages)
        

        
        # Get Job IDs that are on the first page:
        Jobs_on_1st_page = self.find_job_id_aux(soup=soup)
        print("Jobs on first page: ", Jobs_on_1st_page)
        self.List_Job_IDs.extend(Jobs_on_1st_page)
        
        # Iterate over the remaining pages:
        if number_of_pages > 1:
            
            for page_num in range(2,number_of_pages + 1):
                print(f"Scraping page: {page_num}",end="...")
                
                #url = f'https://www.linkedin.com/jobs/search/?keywords={self.job_title}&location={self.location}&start={25 * page_num}'
                #url = requests.utils.requote_uri(url)
                #self.driver.get(url)
                self.scroll_to_load_jobs(page_num=page_num)

                # Parse the HTML content of the page using BeautifulSoup.
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')

                # Get Job Ids present on the page.
                Jobs_on_this_page = self.find_job_id_aux(soup=soup)
                self.List_Job_IDs.extend(Jobs_on_this_page) 
        print("List_Job_IDs: ", self.List_Job_IDs)


    def scrape_job_details(self):
        def remove_tags(html):
            '''remove html tags from BeautifulSoup.text'''
        
            soup = BeautifulSoup(html, "html.parser")
        
            for data in soup(['style', 'script']):
                # Remove tags
                data.decompose()
        
            # return data by retrieving the tag content
            return ' '.join(soup.stripped_strings)
        job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
        job={}
        list_jobs=[]

        for j in range(0, len(self.List_Job_IDs)):
            print(f"{j+1} ... read jobId:{self.List_Job_IDs[j]}")

            resp = requests.get(job_url.format(self.List_Job_IDs[j]))
            soup=BeautifulSoup(resp.text,'html.parser')
            #print(soup.prettify()) 

            job["Job_ID"] = self.List_Job_IDs[j] 
            
            try: 
                job["Job_txt"] = remove_tags(resp.content)
            except:
                job["Job_txt"] = None
            
            try:
                job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
            except:
                job["company"]=None

            try:
                job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
            except:
                job["job-title"]=None

            try:
                job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
            except:
                job["level"]=None

            try:
                job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
            except:
                job["location"]=None

            try:
                job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
            except:
                job["posted-time-ago"]=None

            try:
                nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
                nb_candidats = int(nb_candidats.split()[0])
                job["nb_candidats"]= nb_candidats
            except:
                job["nb_candidats"]=None

            list_jobs.append(job)
            job={}

        # create a pandas Datadrame
        jobs_df = pd.DataFrame(list_jobs)

        return jobs_df

    def scrape_jobs(self):
        self.log_in()
        self.scroll_to_load_jobs()
        self.find_total_job_ids()
        job_df = self.scrape_job_details()
        self.driver.quit()  # Close the WebDriver
        return job_df

In [69]:
if __name__ == "__main__":
    
    # Example usage:
    scraper = LinkedInJobScraper("data scientist", "Ireland")
    #scraper.log_in()
    #scraper.scroll_to_load_jobs()
    #scraper.find_total_job_ids()
    jobs_ireland = scraper.scrape_jobs()
    scraper = LinkedInJobScraper("data scientist", "United Kingdom")
    jobs_uk = scraper.scrape_jobs()
    scraper = LinkedInJobScraper("data scientist", "United States")
    jobs_us = scraper.scrape_jobs()

Entered find_total_job_ids
number_of_jobs: 47
number_of_pages: 2
Jobs on first page:  ['3769507732', '3788682583', '3761125550', '3786718426', '3796122237', '3790056410', '3793012335', '3789727670', '3787695564', '3760686583', '3782387920', '3788291788', '3788673781', '3791922916', '3786708480', '3736663938', '3743276475', '3781139238', '3735137213', '3766974787', '3780068215', '3790376745', '3772710135', '3785604070', '3784409461']
Scraping page: 2...List_Job_IDs:  ['3769507732', '3788682583', '3761125550', '3786718426', '3796122237', '3790056410', '3793012335', '3789727670', '3787695564', '3760686583', '3782387920', '3788291788', '3788673781', '3791922916', '3786708480', '3736663938', '3743276475', '3781139238', '3735137213', '3766974787', '3780068215', '3790376745', '3772710135', '3785604070', '3784409461', '3788677947', '3759300893', '3786178251', '3758742428', '3796868547', '3764250729', '3789579539', '3735134508', '3786182144', '3795203179', '3790310552', '3796851005', '379612473

In [67]:
jobs_dataframe[jobs_dataframe.company.notna()]

,Job_ID,Job_txt,company_name,company,job-title,level,location,posted-time-ago,nb_candidats
2,3761125550,Graduate Programme - Deep Learning Research En...,None,Valeo,Graduate Programme - Deep Learning Research En...,Entry level,"Tuam, County Galway, Ireland",1 week ago,157.0
4,3796122237,"R&D Deep Learning Engineer Valeo Tuam, County ...",None,Valeo,R&D Deep Learning Engineer,Entry level,"Tuam, County Galway, Ireland",None,NaN
5,3790056410,"Data Scientist Kellanova Dublin, County Dublin...",None,Kellanova,Data Scientist,Entry level,"Dublin, County Dublin, Ireland",1 week ago,196.0
7,3787695564,"Data Scientist PTSB Dublin, County Dublin, Ire...",None,PTSB,Data Scientist,Entry level,"Dublin, County Dublin, Ireland",1 week ago,138.0
8,3789727670,Data Scientist - TS/MS Eli Lilly and Company L...,None,Eli Lilly and Company,Data Scientist - TS/MS,Entry level,"Limerick, County Limerick, Ireland",2 weeks ago,74.0
9,3760686583,"Data Scientist, Product Analytics Meta Dublin,...",None,Meta,"Data Scientist, Product Analytics",Not Applicable,"Dublin, County Dublin, Ireland",1 week ago,NaN
12,3788673781,Data Scientist Jacksonstone Recruitment Ltd Li...,None,Jacksonstone Recruitment Ltd,Data Scientist,Associate,"Limerick, County Limerick, Ireland",1 week ago,168.0
13,3791922916,"Data Scientist FRS Recruitment Limerick, Count...",None,FRS Recruitment,Data Scientist,Associate,"Limerick, County Limerick, Ireland",1 week ago,169.0
14,3736663938,"Lead Data Scientist (Bangkok based, relocation...",None,Agoda,"Lead Data Scientist (Bangkok based, relocation...",Mid-Senior level,"Dublin, County Dublin, Ireland",1 week ago,28.0
15,3786708480,"Data Scientist Xcede Dublin, County Dublin, Ir...",None,Xcede,Data Scientist,Entry level,"Dublin, County Dublin, Ireland",1 week ago,136.0


In [74]:
scraper = LinkedInJobScraper("data scientist", "New York")
jobs_us = scraper.scrape_jobs()

Entered find_total_job_ids
number_of_jobs: 281
number_of_pages: 12
Jobs on first page:  ['3785893357', '3782575054', '3725718667', '3774917771', '3787367425', '3774903704', '3725139012', '3765212177', '3780024014', '3694248639', '3790921432', '3586405131', '3746694290', '3786025345', '3782821663', '3770187300', '3773563516', '3742158617', '3784086604', '3774967326', '3788241375', '3788851541', '3766184260', '3746692698', '3786895084']
Scraping page: 2...Scraping page: 3...Scraping page: 4...Scraping page: 5...Scraping page: 6...Scraping page: 7...Scraping page: 8...Scraping page: 9...Scraping page: 10...Scraping page: 11...Scraping page: 12...List_Job_IDs:  ['3785893357', '3782575054', '3725718667', '3774917771', '3787367425', '3774903704', '3725139012', '3765212177', '3780024014', '3694248639', '3790921432', '3586405131', '3746694290', '3786025345', '3782821663', '3770187300', '3773563516', '3742158617', '3784086604', '3774967326', '3788241375', '3788851541', '3766184260', '3746692698

In [75]:
jobs_us

,Job_ID,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats
0,3785893357,"Data Scientist (Algorithms) Middesk New York, ...",Middesk,Data Scientist (Algorithms),Entry level,"New York, United States",2 weeks ago,NaN
1,3782575054,"Data Scientist Stripe New York, United States ...",Stripe,Data Scientist,Mid-Senior level,"New York, United States",3 weeks ago,186.0
2,3725718667,Data Scientist Vail Resorts United States 4 mo...,Vail Resorts,Data Scientist,Entry level,United States,4 months ago,NaN
3,3774917771,,None,None,None,None,None,NaN
4,3787367425,"Data Scientist Point72 New York, United States...",Point72,Data Scientist,Entry level,"New York, United States",2 weeks ago,194.0
...,...,...,...,...,...,...,...,...
273,3796867600,Perl Developer Computershare United States 4 d...,Computershare,Perl Developer,Entry level,United States,4 days ago,NaN
274,3774906497,"Data Scientist, Product Figma United States 1 ...",Figma,"Data Scientist, Product",Mid-Senior level,United States,1 month ago,NaN
275,3796840843,"Senior Manager, Statistical Programming ClickJ...",ClickJobs.io,"Senior Manager, Statistical Programming",Mid-Senior level,"New York, NY",None,NaN
276,3782575053,,None,None,None,None,None,NaN
